# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Dataset

### Overview
The dataset contains the states of tic-tac-toe game and my task is to predict if player x has won or not. By looking at the states. For more info please check readme file.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [1]:
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace

ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'CapstoneProjectAutoML'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EAMJ7AQJK to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [4]:
! pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 28 kB/s s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3


In [6]:
import pandas as pd
df=pd.read_csv('train.csv',index_col=0)
print(df.head())
df.to_csv('trn.csv',index=False)
d=pd.read_csv('trn.csv')
d.head()

   1  2  3  4  5  6  7  8  9         y
0  x  x  x  x  o  o  x  o  o  positive
1  x  x  x  x  o  o  o  x  o  positive
2  x  x  x  x  o  o  o  o  x  positive
3  x  x  x  x  o  o  o  b  b  positive
4  x  x  x  x  o  o  b  o  b  positive


,1,2,3,4,5,6,7,8,9,y
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [7]:
# Import the data
from azureml.core.dataset import Dataset
import pandas as pd


key = 'TicTacToe data'
if key in ws.datasets.keys():
    dataset = ws.datasets[key]
    print("Dataset found")
    datastore = ws.get_default_datastore()
    #train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])

else:
    datastore = ws.get_default_datastore()
    datastore.upload_files(files = ['./trn.csv'])
    train = Dataset.Tabular.from_delimited_files([(datastore,'trn.csv')])
    train = train.register(ws,key)


Uploading an estimated of 1 files
Uploading ./trn.csv
Uploaded ./trn.csv, 1 files out of an estimated total of 1
Uploaded 1 files


# Define Compute instance

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


cpu_cluster_name = "compute-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

Creating a new compute cluster...
Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

This is a classification problem so I chose primary metric as 'accuracy'. The task is not heavy so our automl experment shouldn't require more than 30-40 min, thus exp_timeout min is chosen as 30min.And as we are having 4 nodes I choose 4 max concurent iterations.

In [9]:
from azureml.train.automl import AutoMLConfig

# TODO: Put your automl settings here
automl_settings = {"primary_metric":"accuracy",
                   "experiment_timeout_minutes":30,
                   "enable_early_stopping":True,
                   'n_cross_validations':4,
                   "max_concurrent_iterations": 4}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target = compute_target, task = 'classification', training_data = train, label_column_name ='y',**automl_settings)

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on compute-cluster with default configuration
Running on remote compute: compute-cluster
Parent Run ID: AutoML_5d4d437f-5c73-4b68-abb9-c1951ccd9110

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing v

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
bestRun, bestModel = remote_run.get_output()
model_name = bestRun.properties['model_name']

print('Best model: {}\n\n\nProperties of best model:{}'.format(model_name,bestModel))

Best model: AutoML5d4d437f528


Properties of best model:Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('SparseNormalizer',
                 <azureml.auto...
                                   colsample_bytree=0.6, eta=0.001, gamma=0,
                                   learning_rate=0.1, max_delta_step=0,
                                   max_depth=3, max_leaves=0,
                                   min_child_weight=1, missing=nan,
                                   n_e

In [30]:
bestRun.properties

{'runTemplate': 'automl_child',
 'pipeline_id': '1ad5ca83734cfe5fc2dba8ebf3308814f693d7de',
 'pipeline_spec': '{"objects": [{"class_name": "SparseNormalizer", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"norm": "l2"}, "prepared_kwargs": {}, "spec_class": "preproc"}, {"class_name": "XGBoostClassifier", "module": "automl.client.core.common.model_wrappers", "param_args": [], "param_kwargs": {"booster": "gbtree", "colsample_bytree": 0.6, "eta": 0.001, "max_depth": 3, "max_leaves": 0, "n_estimators": 600, "objective": "reg:logistic", "reg_alpha": 0.10416666666666667, "reg_lambda": 1.6666666666666667, "subsample": 0.7, "tree_method": "auto"}, "prepared_kwargs": {}, "spec_class": "sklearn"}], "pipeline_id": "1ad5ca83734cfe5fc2dba8ebf3308814f693d7de", "module": "sklearn.pipeline", "class_name": "Pipeline", "pipeline_name": "{ SparseNormalizer, XGBoostClassifier }"}',
 'training_percent': '100',
 'predicted_cost': '0.21480199125733757',
 'iteration':

In [13]:
#TODO: Save the best model

model_register= remote_run.register_model(model_name='bestmodel')

model_register.download(target_dir="outputs", exist_ok=True)

'outputs/model.pkl'

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [14]:
# Import Operational Libraries

from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
import requests
import json

In [15]:
config = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=4, 
                                            enable_app_insights=True,
                                            description='bestmodel')

model = Model(ws, 'bestmodel')


# Using the same environment for deployment
env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

Inference_config = InferenceConfig(entry_script="score.py", environment=env_deploy)


# Deploy the best model
service = Model.deploy(workspace=ws, 
                       name='bestmodel', 
                       models=[model], 
                       inference_config=Inference_config, 
                       deployment_config=config)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [16]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-06 02:39:14+00:00 Creating Container Registry if not exists.
2021-04-06 02:39:15+00:00 Registering the environment.
2021-04-06 02:39:16+00:00 Use the existing image.
2021-04-06 02:39:17+00:00 Generating deployment configuration.
2021-04-06 02:39:18+00:00 Submitting deployment to compute.
2021-04-06 02:39:23+00:00 Checking the status of deployment bestmodel..
2021-04-06 02:42:40+00:00 Checking the status of inference endpoint bestmodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [17]:
print(service.scoring_uri)
print(service.swagger_uri)

http://f8ee2cee-2fd4-4fc4-8f2e-211839414d95.southcentralus.azurecontainer.io/score
http://f8ee2cee-2fd4-4fc4-8f2e-211839414d95.southcentralus.azurecontainer.io/swagger.json


In [29]:
# test the web service 

scoring_uri = service.scoring_uri

# Ans should be positive
data={'request':[
    {
        '1':'x',
        '2':'x',
        '3':'b',
        '4':'o',
        '5':'o',
        '6':'o',
        '7':'o',
        '8':'b',
        '9':'b',
    }
]}

# Convert to json string
data=json.dumps(data)
headers = {'Content-Type': 'application/json'}


# Make a request

responce = requests.post(scoring_uri, data, headers=headers)
print(responce.json())

['negative']


TODO: In the cell below, print the logs of the web service and delete the service

In [32]:

print(service.scoring_uri)
print(service.swagger_uri)

http://f8ee2cee-2fd4-4fc4-8f2e-211839414d95.southcentralus.azurecontainer.io/score
http://f8ee2cee-2fd4-4fc4-8f2e-211839414d95.southcentralus.azurecontainer.io/swagger.json


In [31]:

# print the logs
print(service.get_logs())

2021-04-06T02:42:34,444590000+00:00 - iot-server/run 
2021-04-06T02:42:34,447291300+00:00 - gunicorn/run 
2021-04-06T02:42:34,451334300+00:00 - rsyslog/run 
2021-04-06T02:42:34,474278800+00:00 - nginx/run 
rsyslogd: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libuuid.so.1: no version information available (required by rsyslogd)
/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b9c4b27f10281b0e6c0637301639810f/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml

In [ ]:
# delete service
service.delete()

# delete compute cluster
compute-cluster.delete()